<a href="https://colab.research.google.com/github/codecat1111/LLM_Phishing/blob/main/Ollama_Setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run Ollama in Colab
---

[![5aharsh/collama](https://raw.githubusercontent.com/5aharsh/collama/main/assets/banner.png)](https://github.com/5aharsh/collama)

This is an example notebook which demonstrates how to run Ollama inside a Colab instance. With this you can run pretty much any small to medium sized models offerred by Ollama for free.

For the list of available models check [models being offerred by Ollama](https://ollama.com/library).


## Before you proceed
---

Since by default the runtime type of Colab instance is CPU based, in order to use LLM models make sure to change your runtime type to T4 GPU (or better if you're a paid Colab user). This can be done by going to **Runtime > Change runtime type**.

While running your script be mindful of the resources you're using. This can be tracked at **Runtime > View resources**.

## Running the notebook
---

After configuring the runtime just run it with **Runtime > Run all**. And you can start tinkering around. This example uses [Llama 3.2](https://ollama.com/library/llama3.2) to generate a response from a prompted question using [LangChain Ollama Integration](https://python.langchain.com/docs/integrations/chat/ollama/).

## Installing Dependencies
---

1. `pciutils` is required by Ollama to detect the GPU type.
2. Installation of Ollama in the runtime instance will be taken care by `curl -fsSL https://ollama.com/install.sh | sh`




In [1]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
44 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as re

## Running Ollama
---

In order to use Ollama it needs to run as a service in background parallel to your scripts. Becasue Jupyter Notebooks is built to run code blocks in sequence this make it difficult to run two blocks at the same time. As a workaround we will create a service using subprocess in Python so it doesn't block any cell from running.

Service can be started by command `ollama serve`.

`time.sleep(5)` adds some delay to get the Ollama service up before downloading the model.

In [9]:
import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

## Pulling Model
---

Download the LLM model using `ollama pull llama3.2`.

For other models check https://ollama.com/library

In [3]:
!ollama pull llama3.2

## And that's it!
---

With this you should be able to freely play around with the models in your scripts. Following is an example using `langchain-ollama` to answer a simple prompt.

If you have a use-case that can help out others feel free to add your notebook to [Collama](https://github.com/5aharsh/collama/fork)

In [4]:
!pip install langchain-ollama

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from IPython.display import Markdown

template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model="llama3.2")

chain = prompt | model

display(Markdown(chain.invoke({"question": "What's the length of hypotenuse in a right angled triangle"})))

To find the length of the hypotenuse in a right-angled triangle, we can use the Pythagorean theorem.

The Pythagorean theorem states that:

a² + b² = c²

where:
- a is the length of one side (the base or one of the legs)
- b is the length of the other side (the height or the other leg)
- c is the length of the hypotenuse (the longest side, opposite the right angle)

Can I ask which values you'd like to use for a and b?

In [11]:
# If needed in Colab:
# !pip install langchain-core langchain-ollama

import json
import time

from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

# ================== PROMPT + MODEL SETUP ==================

# You can switch to "llama3.2:3b" here if you want the 3B model
model = OllamaLLM(model="llama3.2", temperature=0.0)

template = """
You are a security analyst specializing in phishing email detection.

You MUST respond with a SINGLE valid JSON object only.

Question:
Analyze the following email JSON object and determine if it is legitimate or a phishing attempt.

Output format rules (CRITICAL):
- Output ONLY a JSON object with a single key "classification".
- The value must be:
  - 0 for legitimate
  - 1 for phishing
- Do NOT include explanations, reasoning, comments, Markdown, or any extra text outside the JSON.
- Do NOT wrap the JSON in a code block.

Email Data:
{email_json}

Answer (JSON only):
"""

prompt = ChatPromptTemplate.from_template(template)

# Chain: prompt -> Ollama model
chain = prompt | model

# ================== HELPER: SAFE JSON PARSING ==================

def extract_json_object(text: str):
    """
    Naively extract the first {...} block and parse it as JSON.
    Returns a Python dict, or raises json.JSONDecodeError if parsing fails.
    """
    start = text.find("{")
    end = text.rfind("}")
    if start == -1 or end == -1 or end <= start:
        raise json.JSONDecodeError("No JSON object found", text, 0)

    candidate = text[start : end + 1]
    return json.loads(candidate)

# ================== DATA LOADING ==================

json_filename = "emails_10000_text_only.json"
with open(json_filename, "r") as json_file:
    data = json.load(json_file)

# Ensure the correct key is present
if not all("Email Text" in item for item in data):
    raise KeyError("Key 'Email Text' not found in the JSON file.")

start_time = time.time()  # Start timer
print(f"🔄 Starting classification (LangChain + Ollama): {len(data)} emails to process...\n")

# ================== MAIN LOOP ==================

for index, email_data in enumerate(data, start=1):
    print(f"\n🟢 Processing email {index}/{len(data)}")

    email_start_time = time.time()

    try:
        # Convert email data to JSON string
        email_json = json.dumps(email_data, indent=4)

        # Call the LangChain pipeline (prompt | model)
        llm_output = chain.invoke({"email_json": email_json})

        # For OllamaLLM, invoke() returns a string
        response_content = llm_output if isinstance(llm_output, str) else str(llm_output)

        # Print complete response for logging/debugging
        print("\n📥 Complete Response from LLM:")
        print(response_content)

        try:
            # Safer JSON extraction/parsing
            parsed = extract_json_object(response_content)
            classification = parsed.get("classification", "Unknown")
            email_data["Predicted"] = classification

            # Timing information
            email_time = time.time() - email_start_time
            elapsed_time = time.time() - start_time
            avg_time_per_email = elapsed_time / index
            remaining_emails = len(data) - index
            eta = avg_time_per_email * remaining_emails

            print(f"✅ Email {index} classified as: {classification}")
            print(f"⏱️ Time taken: {email_time:.2f}s - ETA: {eta:.2f}s")

        except json.JSONDecodeError as json_error:
            print(f"⚠️ JSON parsing error for email {index}: {json_error}")
            email_data["Predicted"] = "ParseError"

        except KeyError as key_error:
            print(f"⚠️ Missing key error for email {index}: {key_error}")
            email_data["Predicted"] = "KeyError"

    except Exception as e:
        print(f"❌ Error processing email {index}: {e}")
        email_data["Predicted"] = "Exception"

# ================== SAVE OUTPUT ==================

output_filename = "classified_results_langchain.json"
with open(output_filename, "w") as json_file:
    json.dump(data, json_file, indent=4)

total_time = time.time() - start_time
print(
    f"\n✅ Classification completed in {total_time:.2f}s and saved to '{output_filename}'."
)


Streaming output truncated to the last 5000 lines.
  "products": ["Prepaid MC/Visa cards and other financial services"],
  "partners": ["Transaction Processing Partner", "Leading Brands in Consumer Goods"],
  "managementTeam": ["E. Reese Bogle III, CEO"],
  "strategy": ["Right technology, at the right time, in the right markets"],
  "disclaimer": ["Informational purposes only, not an offer to sell or solicitation of an offer to buy any security"]
}
```
✅ Email 9306 classified as: Unknown
⏱️ Time taken: 6.53s - ETA: 407.54s

🟢 Processing email 9307/10000

📥 Complete Response from LLM:
{"classification":0}
✅ Email 9307 classified as: 0
⏱️ Time taken: 0.61s - ETA: 406.96s

🟢 Processing email 9308/10000

📥 Complete Response from LLM:
{"classification": 1}
✅ Email 9308 classified as: 1
⏱️ Time taken: 0.32s - ETA: 406.35s

🟢 Processing email 9309/10000

📥 Complete Response from LLM:
{"classification": 0}
✅ Email 9309 classified as: 0
⏱️ Time taken: 1.25s - ETA: 405.81s

🟢 Processing email 93